In [46]:
import statsmodels.api as sm
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [47]:
path_url = '/Users/hazelran/Desktop/LCK_SummerPlayoffs_2024.csv'
df_lol = pd.read_csv(path_url, sep=';')

In [48]:
perfect_kda_players = df_lol[df_lol["KDA"] == "Perfect_KDA"]
perf_kda_group = perfect_kda_players.groupby("Player").size().reset_index(name="Perfect KDA Players").sort_values(by="Perfect KDA Players", ascending=False)

condition = df_lol["KDA"] == "Perfect_KDA"
df_lol.loc[condition, "KDA"] = df_lol.loc[condition, "Kills"] + df_lol.loc[condition, "Assists"]
# df_lol[["Player", "Team", "KDA"]].head()

In [49]:
label_encoder = LabelEncoder()
df_lol['Role_encoded'] = label_encoder.fit_transform(df_lol['Role'])

# 导入数据集
X = df_lol[['Role_encoded', 'Kills', 'Deaths', 'Assists', 'KDA', 'Golds', 'CS', 'GPM', 'Dragon Team', 'Baron Team', 'Total damage to Champion', 'Vision Score', 'VSPM', 'Total damage to Champion', 'True Damage', 'KP%', 'Penta kills', 'Damage dealt to turrets', 'Time ccing others', 'Total Time Spent Dead']]
y = df_lol['Total Time CC Dealt']

'\n\n# Standardizing features\nscaler = StandardScaler()\nX_scaled = scaler.fit_transform(X)\n\n# 分割数据集为训练集和测试集\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n\n# Lasso with cross-validation to select the best alpha\nlasso_cv = LassoCV(cv=5, random_state=42).fit(X_train, y_train)\n\n# Predictions\ny_pred = lasso_cv.predict(X_test)\n\n# Model performance\nprint(f"R Squared (Test): {lasso_cv.score(X_test, y_test)}")\nprint(f"Mean Squared Error (Test): {mean_squared_error(y_test, y_pred)}")\n\n# Output coefficients\ncoefficients = pd.DataFrame({\n    \'Feature\': X.columns,\n    \'Coefficient\': lasso_cv.coef_\n})\nprint(coefficients)\n'

In [52]:
"""
from sklearn.metrics import r2_score, mean_squared_error

# 设置Lasso回归模型
lasso = Lasso(alpha=0.1)

# 拟合训练集
lasso.fit(X_train_scaled, y_train)

# 预测测试集
y_pred = lasso.predict(X_test_scaled)

# 计算 R Squared 和 MSE
print(f"R Squared: {r2_score(y_test, y_pred)}")
print(f"Mean Squared Error: {mean_squared_error(y_test, y_pred)}")

# 查看特征的重要性（非零系数的特征）
importance = lasso.coef_
for feature, coef in zip(X.columns, importance):
    print(f"{feature}: {coef}")
"""

R Squared: -0.7215937429011734
Mean Squared Error: 72387.17000910007
Role_encoded: 157.72215142097167
Kills: 36.39796164754343
Deaths: -14.793516188878447
Assists: -64.73789290460425
KDA: 10.263744076838403
Golds: -177.955275028113
CS: 385.7214409007945
GPM: -152.90489702191493
Dragon Team: 96.31976939860255
Baron Team: 43.863403169370486
Total damage to Champion: -4.88898384516154
Vision Score: -34.052211038300314
VSPM: -99.66929329277497
Total damage to Champion: -45.40198193146386
True Damage: 26.723057209639716
KP%: 70.64891956768119
Penta kills: -9.803757636790856
Damage dealt to turrets: -23.303915892920234
Time ccing others: 245.82973328408585
Total Time Spent Dead: -17.16819167121092


Results: Lasso回归模型没有改进，反而导致了更糟糕的性能，特别是R Squared为负数，表示模型没有捕捉到数据的相关性。


错误猜想：可能是因为模型对特征施加了过强的正则化，导致模型缺拟合

In [54]:
# 使用LassoCV进行正则化强度的交叉验证

# Standardizing features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 分割数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Lasso with cross-validation to select the best alpha
lasso_cv = LassoCV(cv=5, random_state=42).fit(X_train, y_train)

# Predictions
y_pred = lasso_cv.predict(X_test)

# Model performance
print(f"R Squared (Test): {lasso_cv.score(X_test, y_test)}")
print(f"Mean Squared Error (Test): {mean_squared_error(y_test, y_pred)}")

# Output coefficients
coefficients = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': lasso_cv.coef_
})
print(coefficients)

R Squared (Test): 0.3122224761850195
Mean Squared Error (Test): 28918.709045105308
                     Feature  Coefficient
0               Role_encoded     0.000000
1                      Kills    -0.000000
2                     Deaths    -0.000000
3                    Assists    -0.000000
4                        KDA     0.000000
5                      Golds     0.032318
6                         CS     1.099310
7                        GPM    -1.613457
8                Dragon Team     0.000000
9                 Baron Team     0.000000
10  Total damage to Champion    -0.003518
11              Vision Score    -4.195229
12                      VSPM    -0.000000
13  Total damage to Champion    -0.005512
14               True Damage     0.025996
15                       KP%     0.000000
16               Penta kills     0.000000
17   Damage dealt to turrets    -0.005613
18         Time ccing others    14.842620
19     Total Time Spent Dead    -0.210847


Results: 从结果来看，Lasso回归模型通过L1正则化自动消除了多余的特征，将一些特征的系数缩小为0。


R Squared(Test): 0.312 --> 表明模型在测试集上的拟合效果较为一般，解释了31%的数据方差


预测误差(MSE: 28918)较大

In [50]:
# 通过网格搜索(GridSearchCV)优化Lasso模型中的alpha值，从而自动调整正则化强度

# 使用交叉验证来选择最佳 alpha
lasso_cv = LassoCV(cv=5, random_state=42)
lasso_cv.fit(X_train, y_train)

# 输出最佳 alpha
print(f"Best alpha: {lasso_cv.alpha_}")

# 使用最佳 alpha 重新训练模型
lasso = Lasso(alpha=lasso_cv.alpha_)
lasso.fit(X_train, y_train)

# 测试集上的预测
y_pred = lasso.predict(X_test)

# 输出新的 R Squared 和 MSE
print(f"R Squared (Test): {lasso.score(X_test, y_test)}")
print(f"Mean Squared Error (Test): {mean_squared_error(y_test, y_pred)}")

Best alpha: 211.760568548387
R Squared (Test): 0.3122224761850195
Mean Squared Error (Test): 28918.709045105308


[Best alpha: 211.760568548387] 表明模型选择了较高的正则化力度。在这个模型中，测试集的R Squared(0.312)表示模型只能解释约31%的数据方差，预测误差(MSE: 28918)较大

In [53]:
# we will store variable/feature names for later
feature_names = X.columns.tolist()

# Ssandardizing features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# constant for the intercept
X_scaled = sm.add_constant(X_scaled)

# using the feature names to include the constant
feature_names = ['const'] + feature_names

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# model fit
model = sm.OLS(y_train, X_train).fit()

# df as the model summary (since the standard summary() does not provide feature/variable names)
summary_df = pd.DataFrame({
    'Feature': feature_names,
    'P>|t|': model.pvalues,
    'Coefficient': model.params,
    'Std Err': model.bse,
    't': model.tvalues,
    '95% Conf. Int. Lower': model.conf_int()[0],
    '95% Conf. Int. Upper': model.conf_int()[1]
})

print("Goodness of Fit stats:")
print(f"R Squared: {model.rsquared.round(3)}")
print(f"Adjusted R Squared: {model.rsquared_adj.round(3)}\n")
print(summary_df)

Goodness of Fit stats:
R Squared: 0.372
Adjusted R Squared: 0.319

                        Feature         P>|t|  Coefficient     Std Err  \
const                     const  1.565557e-31   357.253970   26.098380   
x1                 Role_encoded  1.933495e-06   158.488014   32.433223   
x2                        Kills  6.104191e-01    38.477512   75.419534   
x3                       Deaths  8.353408e-01   -14.596408   70.143086   
x4                      Assists  2.400287e-01   -63.040231   53.515082   
x5                          KDA  8.679940e-01    10.324751   62.049841   
x6                        Golds  2.636426e-01  -179.601728  160.274653   
x7                           CS  2.471281e-02   392.938113  173.801712   
x8                          GPM  1.491757e-01  -155.913003  107.723951   
x9                  Dragon Team  1.607033e-02    94.937570   39.143400   
x10                  Baron Team  2.563260e-01    45.309451   39.815914   
x11    Total damage to Champion  3.118407e-01

小结：使用了Lasso回归来进行特征选择，并比较了不同的模型性能


过程：


1.首先，普通线性回归模型的R²为0.372，虽然部分特征无关紧要，但由于删除高P值的特征导致模型性能变差，R²下降至0.212


2.通过Lasso回归自动选择重要特征，并使用交叉验证确定最佳正则化参数alpha=211.76，测试集的R²为0.312，均方误差为28,918.7


3.虽然R²比初始线性回归模型低，但Lasso回归有效减少了模型的复杂度，提升了模型的泛化能力，降低了过拟合的风险


结论：Lasso回归在选择相关特征方面表现良好，尽管R²有所下降，但它使模型更具泛化性，适合实际应用

Summary: I applied Lasso regression for feature selection and compared it with other models.


Process:


1. Initially, linear regression gave an R² of 0.372. However, removing features with high p-values reduced performance, and R² dropped to 0.212


2. Lasso regression was applied with cross-validation, leading to an optimal alpha of 211.76. The test set yielded an R² of 0.312 with a mean squared error of 28,918.7

3. Although R² was lower compared to the initial model, Lasso simplified the model, improving its generalization and reducing overfitting risk


Conclusion: Lasso regression effectively selected relevant features, resulting in a simpler, more generalizable model, even at the cost of a slightly lower R²
